right now, this compares 9 channel CNN vs. GLMs, in terms of raw performance (cc^2 or cc), on ALL neurons (so no subset stuff yet)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from collections import OrderedDict
from itertools import product
import h5py
import numpy as np
import pandas as pd

In [3]:
from tang_jcompneuro.model_fitting_postprocess import load_data_generic

In [4]:
def check_all(squared, score_col_name):
    models_to_examine = [
        ('cnn', 'b.1'),
        ('cnn', 'b.2'),
        ('cnn', 'b.3'),
        ('cnn', 'b.6'),
        ('cnn', 'b.9'),
        ('glm', 'linear_poisson'),
        ('glm', 'fpower_poisson'),
        ('glm', 'gqm.2_poisson'),
        ('glm', 'gqm.4_poisson'),
        ('glm', 'gqm.8_poisson'),
        ('glm', 'linear_gaussian'),
        ('glm', 'fpower_gaussian'),
        ('glm', 'gqm.2_gaussian'),
        ('glm', 'gqm.4_gaussian'),
        ('glm', 'gqm.8_gaussian'),
    ] # TODO should make this a ordered dict.
      # so that it will be easier to collect best val model for each model class.
    
    return load_data_generic(models_to_examine, load_naive=True, metric='raw', squared=squared,
                             score_col_name=score_col_name, percentages_to_load=(25,50,75,100))

In [5]:
df_all_cc = check_all(squared=False, score_col_name='cc')

MkA_Shape all
MkA_Shape OT
MkE2_Shape all
MkE2_Shape OT


In [6]:
df_all_cc2 = check_all(squared=True, score_col_name='cc2')

MkA_Shape all
MkA_Shape OT
MkE2_Shape all
MkE2_Shape OT


In [7]:
# this is the way to change a multiindex's names.
# df_all.index.rename(['haha','haha2','haha3'], inplace=True)

In [8]:
df_all = pd.concat([df_all_cc, df_all_cc2], axis=1)
df_all.columns.name = 'metric'

In [9]:
df_all.unstack('subset').unstack('percentage')

metric                                            cc                      \
subset                                            OT                       
percentage                                       25        50        75    
dataset    model                                                           
MkA_Shape  cnn_b.1$False/False              0.203482  0.255579  0.284343   
           cnn_b.2$False/False              0.222258  0.278473  0.303727   
           cnn_b.3$False/False              0.228465  0.282565  0.311184   
           cnn_b.6$False/False              0.236207  0.290970  0.318146   
           cnn_b.9$False/False              0.240353  0.295794  0.321300   
           glm_fpower_gaussian$False/False  0.196035  0.233797  0.249867   
           glm_fpower_poisson$False/False   0.197372  0.233346  0.251369   
           glm_gqm.2_gaussian$False/False   0.152520  0.191270  0.214334   
           glm_gqm.2_poisson$False/False    0.156291  0.198711  0.223213   
           glm_gqm.4_gaussian$False/False   0.153093  0.200406  0.224381   
           glm_gqm.4_poisson$False/False    0.157559  0.207038  0.232276   
           glm_gqm.8_gaussian$False/False   0.144163  0.192944  0.217162   
           glm_gqm.8_poisson$False/False    0.144598  0.195143  0.221188   
           glm_linear_gaussian$False/False  0.147168  0.171281  0.184606   
           glm_linear_poisson$False/False   0.153542  0.175858  0.192788   
MkE2_Shape cnn_b.1$False/False              0.236411  0.309676  0.345027   
           cnn_b.2$False/False              0.267294  0.343642  0.375883   
           cnn_b.3$False/False              0.280250  0.350106  0.384243   
           cnn_b.6$False/False              0.293922  0.365261  0.398304   
           cnn_b.9$False/False              0.298399  0.366499  0.399061   
           glm_fpower_gaussian$False/False  0.224667  0.268426  0.289003   
           glm_fpower_poisson$False/False   0.223642  0.267847  0.288741   
           glm_gqm.2_gaussian$False/False   0.180987  0.246647  0.275221   
           glm_gqm.2_poisson$False/False    0.178064  0.257111  0.286107   
           glm_gqm.4_gaussian$False/False   0.179890  0.249780  0.283408   
           glm_gqm.4_poisson$False/False    0.177469  0.252427  0.289912   
           glm_gqm.8_gaussian$False/False   0.162252  0.237284  0.270367   
           glm_gqm.8_poisson$False/False    0.156002  0.234865  0.271772   
           glm_linear_gaussian$False/False  0.184443  0.226468  0.244978   
           glm_linear_poisson$False/False   0.189885  0.238631  0.259594   

metric                                                                    \
subset                                                     all             
percentage                                       100       25        50    
dataset    model                                                           
MkA_Shape  cnn_b.1$False/False              0.300747  0.297344  0.334111   
           cnn_b.2$False/False              0.319944  0.329177  0.372185   
           cnn_b.3$False/False              0.325641  0.339392  0.384259   
           cnn_b.6$False/False              0.333065  0.352292  0.396103   
           cnn_b.9$False/False              0.338384  0.358187  0.400845   
           glm_fpower_gaussian$False/False  0.260039  0.249654  0.272533   
           glm_fpower_poisson$False/False   0.261663  0.248455  0.274507   
           glm_gqm.2_gaussian$False/False   0.221168  0.247768  0.288095   
           glm_gqm.2_poisson$False/False    0.231374  0.257239  0.301228   
           glm_gqm.4_gaussian$False/False   0.235860  0.261707  0.303113   
           glm_gqm.4_poisson$False/False    0.244452  0.271202  0.315092   
           glm_gqm.8_gaussian$False/False   0.234528  0.257300  0.297860   
           glm_gqm.8_poisson$False/False    0.237862  0.265291  0.308186   
           glm_linear_gaussian$False/False  0.193430  0.195722  0.212475   
           glm_linear_poisson$False/False   0.2